In [15]:
import numpy as np
import pandas as pd
import swifter
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
import re
import stanfordnlp
from spacy_stanfordnlp import StanfordNLPLanguage
import pandas as pd
snlp = stanfordnlp.Pipeline(lang="en")
nlp = StanfordNLPLanguage(snlp)
import warnings
warnings.filterwarnings("ignore")
from Extractor import Extractor

Use device: gpu
---
Loading: tokenize
With settings: 
{'model_path': 'C:\\Users\\DRAGooN\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tokenizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: pos
With settings: 
{'model_path': 'C:\\Users\\DRAGooN\\stanfordnlp_resources\\en_ewt_models\\en_ewt_tagger.pt', 'pretrain_path': 'C:\\Users\\DRAGooN\\stanfordnlp_resources\\en_ewt_models\\en_ewt.pretrain.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
---
Loading: lemma
With settings: 
{'model_path': 'C:\\Users\\DRAGooN\\stanfordnlp_resources\\en_ewt_models\\en_ewt_lemmatizer.pt', 'lang': 'en', 'shorthand': 'en_ewt', 'mode': 'predict'}
Building an attentional Seq2Seq model...
Using a Bi-LSTM encoder
Using soft attention for LSTM.
Finetune all embeddings.
[Running seq2seq lemmatizer with edit classifier]
---
Loading: depparse
With settings: 
{'model_path': 'C:\\Users\\DRAGooN\\stanfordnlp_resources\\en_ewt_models\\en_ewt_parser.pt', 'pretrain_path': 'C:\\Use

In [3]:
def sigullarize(list_string, mode = "list"):
    #Fettuccine has toppings of Creamy Alfredo and House-Roasted Chicken")
    #print(token.text, token.lemma_)
    if mode=="list":
        for doc in list_string:
            doc = nlp(doc)
        return [wrd.lemma_ for wrd in doc]
    else:
        doc = nlp(list_string)
        return [wrd.lemma_ for wrd in doc][0]

### Data we have prepared

* Food Attributes
* ingridients

In [5]:
attributes_df = pd.read_excel("./data/product_attributes.xlsx")
attributes_df['Values_mod'] = attributes_df['Values'].str.lower().str.strip().str.split(r', +|,| and | or | ?& ?|\.|^ $')
attributes_df['Attribute_mod'] = attributes_df.swifter.apply(lambda x: sigullarize(x['Attribute'], mode='word'), axis =1)
attributes_df['Item_mod'] = attributes_df['Item_mod'].str.lower()#.swifter.apply(lambda x: sigullarize(x['Item'], mode='word'), axis =1)
attributes_df['Item']  = attributes_df['Item'] .str.lower().str.strip()
attributes_df.head().style

,Item_mod,Item,Attribute,Values,Values_mod,Attribute_mod
0,pasta,pastas,Type,"Spaghetti, Fettuccine, Ziti, Tortellini, Rotini or Ravioli, Penne, Lasagna, Lasagne","['spaghetti', 'fettuccine', 'ziti', 'tortellini', 'rotini', 'ravioli', 'penne', 'lasagna', 'lasagne']",type
1,pasta,pastas,Sauce,"Meat Sauce, Marinara Sauce, Alfredo, Spicy Tomato Pepper Sauce, Creamy Basil","['meat sauce', 'marinara sauce', 'alfredo', 'spicy tomato pepper sauce', 'creamy basil']",sauce
2,pasta,pastas,Cheese,"Provolone, Mozzarella, Parmesan, Ricotta, Romano","['provolone', 'mozzarella', 'parmesan', 'ricotta', 'romano']",cheese
3,pasta,pastas,Topping,"Italian Meatballs, Grape Tomatoes, Sausage, Garlic-Roasted Mushrooms, Bacon, peas, black pepper, House-Roasted Chicken, Roasted Chicken, Fresh Spinach, Broccoli, Fire-Roasted Tomatoes, Pepperoni","['italian meatballs', 'grape tomatoes', 'sausage', 'garlic-roasted mushrooms', 'bacon', 'peas', 'black pepper', 'house-roasted chicken', 'roasted chicken', 'fresh spinach', 'broccoli', 'fire-roasted tomatoes', 'pepperoni']",top
4,salad,salads,Type,Vegetable and Non Vegetable.,"['vegetable', 'non vegetable', '']",type


In [6]:
attributes = attributes_df.apply(lambda x: pd.Series(x['Values_mod']), axis=1).stack().reset_index(level=1, drop=True)
attributes.name = 'Values_mod'
attrib = attributes_df.drop('Values_mod', axis=1).join(attributes)
attrib.index = attrib.Values_mod
attrib.to_csv("attrib.csv")
attrib.head()

,Item_mod,Item,Attribute,Values,Attribute_mod,Values_mod
Values_mod,,,,,,
spaghetti,pasta,pastas,Type,"Spaghetti, Fettuccine, Ziti, Tortellini, Rotin...",type,spaghetti
fettuccine,pasta,pastas,Type,"Spaghetti, Fettuccine, Ziti, Tortellini, Rotin...",type,fettuccine
ziti,pasta,pastas,Type,"Spaghetti, Fettuccine, Ziti, Tortellini, Rotin...",type,ziti
tortellini,pasta,pastas,Type,"Spaghetti, Fettuccine, Ziti, Tortellini, Rotin...",type,tortellini
rotini,pasta,pastas,Type,"Spaghetti, Fettuccine, Ziti, Tortellini, Rotin...",type,rotini


In [7]:
## Items present in the list
list(set(attrib['Item'].values))

['subway',
 'desserts',
 'burgers',
 'pastas',
 'drinks',
 'sandwich',
 'salads',
 'pizzas',
 'wraps']

In [8]:
## Attributes of Item = Pasta
df = attrib
i= "pasta"
my_list = filter(lambda x: x != '', list(df[(df['Item']==i) | (df['Item_mod']==i)]['Values_mod']))
set(my_list)

{'alfredo',
 'bacon',
 'black pepper',
 'broccoli',
 'creamy basil',
 'fettuccine',
 'fire-roasted tomatoes',
 'fresh spinach',
 'garlic-roasted mushrooms',
 'grape tomatoes',
 'house-roasted chicken',
 'italian meatballs',
 'lasagna',
 'lasagne',
 'marinara sauce',
 'meat sauce',
 'mozzarella',
 'parmesan',
 'peas',
 'penne',
 'pepperoni',
 'provolone',
 'ravioli',
 'ricotta',
 'roasted chicken',
 'romano',
 'rotini',
 'sausage',
 'spaghetti',
 'spicy tomato pepper sauce',
 'tortellini',
 'ziti'}

In [10]:
ingridients = pd.read_excel("./data/Food_ingredients_list_26k.xlsx")
ingridients['Item_mod'] = ingridients['Item'].str.lower().str.strip()
print(ingridients.head(),"\n", ingridients.tail())

        Item   Item_mod
0    Oranges    oranges
1    Bananas    bananas
2     Apples     apples
3  Kiwifruit  kiwifruit
4   Sultanas   sultanas 
                           Item                 Item_mod
26395               Mayonnaise               mayonnaise
26396        Dried mixed herbs        dried mixed herbs
26397  Salted blanched peanuts  salted blanched peanuts
26398     Sliced or shaved ham     sliced or shaved ham
26399              Chewing gum              chewing gum


In [16]:
obj = Extractor(attrib, ingridients)

### Example

Here for the text ***Fettuccine has toppings of Creamy Alfredo and House-Roasted Chicken***

We need to attribute to Pasta ==> Attributes  ==>
<br> 
**{ <br>
                                                    "sauce": "Alfredo",<br>
                                                    "topping":"House-Roasted Chicken",<br>
                                                    "type": "Fettuccine"<br>
                                                    }**

In [17]:
output_json = obj.process_input('Fettuccine Pasta has toppings of Creamy Alfredo and House-Roasted Chicken')
for i in output_json:
    print(i,':',output_json[i], "\n")

No common elements
alfredo : Sauce 

house-roasted chicken : Topping 

fettuccine : Type 

items : ['pasta'] 



In [18]:
df = pd.read_csv('Fazoli_DoorDash_Austin_78717__.csv', skiprows=3)
# df = df.set_index('id')
df.columns = map(str.lower, df.columns)
df.head()

,name,id,category,description,price,calories,suggested item
0,Apple Juice,104413287.0,Drinks and Italian Ice,,1.49,80.0,"['Roasted Chicken', 'Broccoli', 'Triple Chocol..."
1,Baked Chicken Parmigiano,104413244.0,Popular Items/Signature Pasta Bakes,Spaghetti covered in Marinara Sauce with Crisp...,8.49,690.0,"['Italian Sausage', 'Extra Breadsticks (Dozen)..."
2,Baked Lasagna,104413278.0,Popular Items/Fazoli's Fan Favorites,Lasagna layered with Meat Sauce and Mozzarella...,8.49,630.0,"['Broccoli', 'Extra Breadsticks (Dozen)', 'Ext..."
3,Baked Spaghetti,104413277.0,Fazoli's Fan Favorites,Spaghetti with Marinara Sauce loaded with Mozz...,7.49,470.0,"['Meatballs', 'Extra Breadsticks (Dozen)', 'Ex..."
4,Baked Spaghetti with Meatballs,104413281.0,Fazoli's Fan Favorites,Spaghetti with Marinara Sauce loaded with Mozz...,8.49,690.0,"['Triple Chocolate Cheesecake', 'Caesar Side S..."


In [19]:
df['Input'] = df['category'].fillna('')+" "+df['name'].fillna('')+" "+ df['description'].fillna('')
df['Input'] = df['Input'].astype('str')
df['output_json'] = df.swifter.apply(lambda x: obj.process_input(x['Input']), axis =1)

No common elements
No common elements
No common elements
[]
['lasagna', 'mozzarella', 'provolone']
	lasagna
	mozzarella
	provolone
No such item found
No common elements
No common elements
No common elements
No common elements
No common elements
No common elements
No common elements
No common elements
[]
['mozzarella', 'spaghetti']
	mozzarella
	spaghetti
No such item found
No common elements
No common elements
No common elements
No common elements
No common elements
No common elements
No common elements
No common elements
[]
['mozzarella', 'spaghetti', 'meatballs']
	mozzarella
	spaghetti
	meatballs
No such item found
No common elements
No common elements
No common elements
No common elements
No common elements
No common elements
No common elements
No common elements
[]
['ziti', 'mozzarella', 'penne', 'provolone']
	ziti
	mozzarella
	penne
	provolone
No such item found
No common elements
No common elements
No common elements
No common elements
No common elements
No common elements
No comm

In [20]:
df.style

,name,id,category,description,price,calories,suggested item,Input,output_json
0,Apple Juice,1.04413e+08,Drinks and Italian Ice,,1.49,80,"['Roasted Chicken', 'Broccoli', 'Triple Chocolate Cheesecake']",Drinks and Italian Ice Apple Juice,{'items': ['drinks']}
1,Baked Chicken Parmigiano,1.04413e+08,Popular Items/Signature Pasta Bakes,"Spaghetti covered in Marinara Sauce with Crispy Chicken Breast covered with more Marinara Sauce, Mozzarella and Provolone Cheeses and baked.",8.49,690,"['Italian Sausage', 'Extra Breadsticks (Dozen)', 'Extra Breadsticks (Half Dozen)']","Popular Items/Signature Pasta Bakes Baked Chicken Parmigiano Spaghetti covered in Marinara Sauce with Crispy Chicken Breast covered with more Marinara Sauce, Mozzarella and Provolone Cheeses and baked.","{'mozzarella': 'Cheese', 'spaghetti': 'Type', 'provolone': 'Cheese', 'marinara sauce': 'Sauce', 'ingridients': ['chicken breast'], 'items': ['pasta']}"
2,Baked Lasagna,1.04413e+08,Popular Items/Fazoli's Fan Favorites,Lasagna layered with Meat Sauce and Mozzarella and Provolone Cheeses.,8.49,630,"['Broccoli', 'Extra Breadsticks (Dozen)', 'Extra Breadsticks (Half Dozen)']",Popular Items/Fazoli's Fan Favorites Baked Lasagna Lasagna layered with Meat Sauce and Mozzarella and Provolone Cheeses.,"{'lasagna': 'Type', 'mozzarella': 'Cheese', 'provolone': 'Cheese', 'items': 'Unkown'}"
3,Baked Spaghetti,1.04413e+08,Fazoli's Fan Favorites,"Spaghetti with Marinara Sauce loaded with Mozzarella Cheese and baked to a golden, bubbly perfection. Vegetarian friendly.",7.49,470,"['Meatballs', 'Extra Breadsticks (Dozen)', 'Extra Breadsticks (Half Dozen)']","Fazoli's Fan Favorites Baked Spaghetti Spaghetti with Marinara Sauce loaded with Mozzarella Cheese and baked to a golden, bubbly perfection. Vegetarian friendly.","{'mozzarella': 'Cheese', 'spaghetti': 'Type', 'items': 'Unkown'}"
4,Baked Spaghetti with Meatballs,1.04413e+08,Fazoli's Fan Favorites,"Spaghetti with Marinara Sauce loaded with Mozzarella Cheese and baked to a golden, bubbly perfection topped with Meatballs.",8.49,690,"['Triple Chocolate Cheesecake', 'Caesar Side Salad', 'Extra Breadsticks (Half Dozen)']","Fazoli's Fan Favorites Baked Spaghetti with Meatballs Spaghetti with Marinara Sauce loaded with Mozzarella Cheese and baked to a golden, bubbly perfection topped with Meatballs.","{'mozzarella': 'Cheese', 'spaghetti': 'Type', 'meatballs': 'Toppings', 'items': 'Unkown'}"
5,Baked Ziti,1.04413e+08,Fazoli's Fan Favorites,"Penne with Marinara and Meat Sauce topped with Mozzarella and Provolone Cheeses and baked to a golden, bubbly perfection.",7.99,570,"['Triple Chocolate Cheesecake', 'Extra Breadsticks (Dozen)', 'Extra Breadsticks (Half Dozen)']","Fazoli's Fan Favorites Baked Ziti Penne with Marinara and Meat Sauce topped with Mozzarella and Provolone Cheeses and baked to a golden, bubbly perfection.","{'ziti': 'Type', 'mozzarella': 'Cheese', 'penne': 'Type', 'provolone': 'Cheese', 'items': 'Unkown'}"
6,Bottled Water,1.04413e+08,Drinks and Italian Ice,nan,2.29,nan,"['Italian Sausage', 'Chicken Bacon Caesar', 'Roasted Chicken']",Drinks and Italian Ice Bottled Water,"{'ingridients': ['bottled water'], 'items': ['drinks']}"
7,Broccoli,1.04413e+08,Sides and Extras,nan,1.5,nan,"['Chicken Bacon Caesar', 'Garlic-Roasted Mushrooms', 'Meatballs']",Sides and Extras Broccoli,"{'broccoli': 'Topping', 'ingridients': ['broccoli'], 'items': 'Unkown'}"
8,Caesar Side Salad,1.04413e+08,Salads,"Parmesan croutons, Cesar dressing.",2.79,80,"['Chocolate Chunk Brownie', 'Extra Breadsticks (Dozen)', 'Extra Breadsticks (Half Dozen)']","Salads Caesar Side Salad Parmesan croutons, Cesar dressing.","{'caesar': 'Dressing', 'items': ['salads', 'salad']}"
9,Chicken Bacon Caesar,1.04413e+08,Salads,Parmesan romaine chicken bacon grope tomato Caesar dressing.,7.89,620,"['Extra Breadsticks (Dozen)', 'Chocolate Chunk Brownie', 'Extra Breadsticks (Half Dozen)']",Salads Chicken Bacon Caesar Parmesan romaine chicken bacon grope tomato Caesar dressing.,"{'caesar':

In [21]:
df.to_csv("Output_key_words.csv", index = False)